In [1]:
!pip install databricks-sql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pystache: filename=pystache-0.6.0-py3-none-any.whl size=83618 sha256=4d225684bd3e17639b81b578cfcea7340d4e1516769e58e84d861f34ecb1472b
  Stored in directory: /root/.cache/pip/wheels/4a/98/2c/84a50139380b5a5d8c0b4dfb92f2d860953c9e8e755507a402
  Created wheel for thrift: filename=thrift-0.16.0-cp310-cp310-linux_x86_64.whl size=373866 sha256=a55ae0a5a68e63524b34a4668bf2707cac9dfc77a39c5a8bf0b8ee8d707255

Importing User Data

In [2]:
from databricks import sql
import pandas as pd

with sql.connect(server_hostname = "hostname",
                 http_path       = "http-path",
                 access_token    = "access-token") as connection:

  with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM flight_recommendation.default.user")
    result = cursor.fetchall()
    user_data = pd.DataFrame(result)

In [3]:
user_data.columns = ['userid','name','age','email','phone_number','travel_purpose','flight_id','id']

In [4]:
user_data.head()

,userid,name,age,email,phone_number,travel_purpose,flight_id,id
0,1,Melissa Terry,22,jenniferwhite@example.com,+9749697939599,Education,5826,0
1,1,Melissa Terry,22,jenniferwhite@example.com,+9749697939599,Education,4661,1
2,1,Melissa Terry,22,jenniferwhite@example.com,+9749697939599,Education,5281,2
3,2,Aaron Harris,27,tyler31@example.org,+513359988596,Other,2131,3
4,2,Aaron Harris,27,tyler31@example.org,+513359988596,Other,2479,4


In [5]:
user_data.shape

(9936, 8)

In [6]:
user_data['userid'] = user_data['userid'].astype(int)
user_data['flight_id'] = user_data['flight_id'].astype(int)

In [7]:
user_data.dtypes

userid             int64
name              object
age               object
email             object
phone_number      object
travel_purpose    object
flight_id          int64
id                 int64
dtype: object

Importing Flight Data

In [19]:
with sql.connect(server_hostname = "hostname",
                 http_path       = "http-path",
                 access_token    = "access-token") as connection:

  with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM flight_recommendation.default.flight")
    result = cursor.fetchall()
    flight_data = pd.DataFrame(result)

In [20]:
flight_data.columns = ['id','_c0','to','from','date_to','date_from','pos_overall','url','type','price',
'airline','duration']

In [26]:
flight_data.dtypes

id               int64
_c0             object
to              object
from            object
date_to         object
date_from       object
pos_overall    float64
url             object
type            object
price            int64
airline         object
duration        object
dtype: object

In [22]:
flight_data['pos_overall'] = flight_data['pos_overall'].astype(float)


In [25]:
flight_data['price'] = flight_data['price'].astype(int)

In [23]:
flight_data.head()

,id,_c0,to,from,date_to,date_from,pos_overall,url,type,price,airline,duration
0,0,0,"Phoenix, AZ (all airports)","New York, NY (all airports)",2023-12-16,2023-12-10,1.0,https://www.google.com/travel/flights?sca_esv=...,Nonstop,258,Spirit,5h 42m
1,1,1,"Phoenix, AZ (all airports)","New York, NY (all airports)",2023-12-16,2023-12-10,1.0,https://www.google.com/travel/flights?sca_esv=...,Nonstop,358,American,5h 46m
2,2,2,"Phoenix, AZ (all airports)","New York, NY (all airports)",2023-12-16,2023-12-10,1.0,https://www.google.com/travel/flights?sca_esv=...,Nonstop,358,Delta,5h 54m
3,3,3,"Phoenix, AZ (all airports)","New York, NY (all airports)",2023-12-16,2023-12-10,1.0,https://www.google.com/travel/flights?sca_esv=...,Connecting,412,Alaska,10h 15m+
4,4,4,"Phoenix, AZ (all airports)","New York, NY (all airports)",2023-12-18,2023-12-11,1.0,https://www.google.com/travel/flights?sca_esv=...,Nonstop,228,Spirit,5h 42m


Importing Flight Popularity Data

In [28]:
with sql.connect(server_hostname = "hostname",
                 http_path       = "http-path",
                 access_token    = "access-token") as connection:

  with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM flight_recommendation.default.flightpopularity")
    result = cursor.fetchall()
    data = pd.DataFrame(result)

In [29]:
data.columns = ['airline','popularity_score']

In [30]:
data.head()

,airline,popularity_score
0,Spirit,1375
1,American,784
2,Delta,544
3,Alaska,158
4,JetBlue,338


Recommend Flights to New User

In [31]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

def generate_recommendations_extended(flight_data, popularity_data, price_range,
                                      date_range,
                                      source, destination):
    # Assuming flight_data is a Pandas DataFrame with columns: 'id', 'airline', 'price', 'date', 'source', 'destination'
    # Convert the 'price' column to numeric values
       # Filter flights based on user inputs

    date_mask = (flight_data['date_from'] >= date_range[0]) & (flight_data['date_to'] <= date_range[1])
    source_dest_mask = ((flight_data['from'].str.contains(source)) & (flight_data['to'].str.contains(destination)))
    price_mask = (flight_data['price'] >= price_range[0]) & (flight_data['price'] <= price_range[1])

    filtered_data = flight_data[source_dest_mask & date_mask & price_mask]

    # Merge with popularity data and sort by popularity
    if not filtered_data.empty:
        recommendations = pd.merge(filtered_data, popularity_data, how='left', on='airline')
        #recommendations = recommendations[source_dest_mask]
        recommendations = recommendations.sort_values(by='popularity_score', ascending=False)

    return recommendations


# Example usage:
price_range = (300, 500)

date_range = ('2023-12-10', '2023-12-18')  # Adjust the date range as needed
source = 'New York'  # Replace with the actual source
destination = 'Los Angeles'  # Replace with the actual destination

result = generate_recommendations_extended(flight_data, data , price_range ,date_range,
                                             source, destination)
flight_result = pd.DataFrame(result)
flight_result


,id,_c0,to,from,date_to,date_from,pos_overall,url,type,price,airline,duration,popularity_score
2,357,379,"Los Angeles, CA (LAX)","New York, NY (all airports)",2023-12-18,2023-12-11,1.0,https://www.google.com/travel/flights?sca_esv=...,Connecting,420,Spirit,1d 0h+,1375.0
4,361,383,"Los Angeles, CA (LAX)","New York, NY (all airports)",2023-12-18,2023-12-12,1.0,https://www.google.com/travel/flights?sca_esv=...,Connecting,427,Spirit,18h 24m+,1375.0
5,362,384,"Los Angeles, CA (LAX)","New York, NY (all airports)",2023-12-18,2023-12-12,1.0,https://www.google.com/travel/flights?sca_esv=...,Nonstop,438,Spirit,6h 16m,1375.0
0,352,374,"Los Angeles, CA (LAX)","New York, NY (all airports)",2023-12-18,2023-12-10,1.0,https://www.google.com/travel/flights?sca_esv=...,Connecting,498,Multiple airlines,1d 2h+,NaN
1,356,378,"Los Angeles, CA (LAX)","New York, NY (all airports)",2023-12-18,2023-12-11,1.0,https://www.google.com/travel/flights?sca_esv=...,Connecting,399,Multiple airlines,1d 0h+,NaN
3,360,382,"Los Angeles, CA (LAX)","New York, NY (all airports)",2023-12-18,2023-12-12,1.0,https://www.google.com/travel/flights?sca_esv=...,Connecting,405,Multiple airlines,12h 33m+,NaN


#WORK FROM HERE

In [32]:
from sklearn.metrics.pairwise import cosine_similarity

def generate_flight_recommendations(user_id, flight_data, user_data, price_range, date_range, source, destination,top_k=3):
    # Merge flight and user data
    merged_data = pd.merge(flight_data, user_data, how='left', left_on='id', right_on='flight_id')

    merged_data['userid'] = merged_data['userid'].astype(float).fillna(0)
    merged_data['flight_id'] = merged_data['flight_id'].astype(float).fillna(0)
    merged_data['flight_id'] = merged_data['flight_id'].astype(int)

    # Filter flights for the given user
    user_flights = merged_data[merged_data['userid'] == user_id]

    # Create a user profile based on historical airlines
    user_profile = user_flights.groupby('airline').size().reset_index(name='num_trips')

    # Content-Based Filtering: Recommend flights based on historical airlines
    recommended_airlines = user_flights['airline'].value_counts().index[:top_k].tolist()

    # Get entire flight data for the recommended airlines
    recommended_flights_data = flight_data[flight_data['airline'].isin(recommended_airlines)]

    # Sort recommended flights based on the count of historical flights for each airline
    recommended_flights_data['airline_count'] = recommended_flights_data['airline'].map(user_profile.set_index('airline')['num_trips'])
    recommended_flights_data = recommended_flights_data.sort_values(by='airline_count', ascending=False).drop('airline_count', axis=1)

    date_mask = (recommended_flights_data['date_from'] >= date_range[0]) & (recommended_flights_data['date_to'] <= date_range[1])
    source_dest_mask = ((recommended_flights_data['from'].str.contains(source)) & (recommended_flights_data['to'].str.contains(destination)))

    price_mask = (recommended_flights_data['price'].astype(int) >= price_range[0]) & (recommended_flights_data['price'].astype(int) <= price_range[1])


    recommend_flight = recommended_flights_data[date_mask & price_mask & source_dest_mask]
    if recommend_flight.empty:
      date_mask = (flight_data['date_from'] >= date_range[0]) & (flight_data['date_to'] <= date_range[1])
      source_dest_mask = ((flight_data['from'].str.contains(source)) & (flight_data['to'].str.contains(destination)))

      price_mask = (flight_data['price'].astype(int) >= price_range[0]) & (flight_data['price'].astype(int) <= price_range[1])

      filtered_data = flight_data[date_mask & price_mask & source_dest_mask]
      return pd.DataFrame(filtered_data)
    else:

      # get all flights data
      date_mask = (flight_data['date_from'] >= date_range[0]) & (flight_data['date_to'] <= date_range[1])
      source_dest_mask = ((flight_data['from'].str.contains(source)) & (flight_data['to'].str.contains(destination)))

      price_mask = (flight_data['price'].astype(int) >= price_range[0]) & (flight_data['price'].astype(int) <= price_range[1])

      filtered_data_1 = flight_data[date_mask & price_mask & source_dest_mask]

      # get flights data from present in user history
      date_mask = (recommend_flight['date_from'] >= date_range[0]) & (recommend_flight['date_to'] <= date_range[1])
      source_dest_mask = ((recommend_flight['from'].str.contains(source)) & (recommend_flight['to'].str.contains(destination)))

      price_mask = (recommend_flight['price'].astype(int) >= price_range[0]) & (recommend_flight['price'].astype(int) <= price_range[1])

      filtered_data_2 = recommend_flight[date_mask & price_mask & source_dest_mask]

      filtered_data = pd.concat([filtered_data_1, filtered_data_2])
      filtered_data = filtered_data.drop_duplicates(subset=['id'])
      return pd.DataFrame(filtered_data)

# Example usage
user_id_to_recommend = 1
price_range = (0, 500)

date_range = ('2023-12-10', '2023-12-20')  # Adjust the date range as needed
source = 'New York'  # Replace with the actual source
destination = 'Los Angeles'  # Replace with the actual destination
recommended_flights_data = generate_flight_recommendations(user_id_to_recommend, flight_data, user_data,
                                                           price_range, date_range, source, destination)
print("Recommended Flights Data:")
recommended_flights_data


Recommended Flights Data:


<ipython-input-32-cce306b120b7>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_flights_data['airline_count'] = recommended_flights_data['airline'].map(user_profile.set_index('airline')['num_trips'])


,id,_c0,to,from,date_to,date_from,pos_overall,url,type,price,airline,duration
352,352,374,"Los Angeles, CA (LAX)","New York, NY (all airports)",2023-12-18,2023-12-10,1.0,https://www.google.com/travel/flights?sca_esv=...,Connecting,498,Multiple airlines,1d 2h+
356,356,378,"Los Angeles, CA (LAX)","New York, NY (all airports)",2023-12-18,2023-12-11,1.0,https://www.google.com/travel/flights?sca_esv=...,Connecting,399,Multiple airlines,1d 0h+
357,357,379,"Los Angeles, CA (LAX)","New York, NY (all airports)",2023-12-18,2023-12-11,1.0,https://www.google.com/travel/flights?sca_esv=...,Connecting,420,Spirit,1d 0h+
360,360,382,"Los Angeles, CA (LAX)","New York, NY (all airports)",2023-12-18,2023-12-12,1.0,https://www.google.com/travel/flights?sca_esv=...,Connecting,405,Multiple airlines,12h 33m+
361,361,383,"Los Angeles, CA (LAX)","New York, NY (all airports)",2023-12-18,2023-12-12,1.0,https://www.google.com/travel/flights?sca_esv=...,Connecting,427,Spirit,18h 24m+
362,362,384,"Los Angeles, CA (LAX)","New York, NY (all airports)",2023-12-18,2023-12-12,1.0,https://www.google.com/travel/flights?sca_esv=...,Nonstop,438,Spirit,6h 16m
364,364,386,"Los Angeles, CA (LAX)","New York, NY (all airports)",2023-12-19,2023-12-13,1.0,https://www.google.com/travel/flights?sca_esv=...,Connecting,375,Multiple airlines,9h 35m+
365,365,387,"Los Angeles, CA (LAX)","New York, NY (all airports)",2023-12-19,2023-12-13,1.0,https://www.google.com/travel/flights?sca_esv=...,Connecting,403,Spirit,9h 35m+
366,366,388,"Los Angeles, CA (LAX)","New York, NY (all airports)",2023-12-19,2023-12-13,1.0,https://www.google.com/travel/flights?sca_esv=...,Nonstop,478,Spirit,6h 16m
